In [1]:
import os

import sqlite3
import pandas

In [2]:
# open database connection
connection = sqlite3.connect('data/l1000.db')
cursor = connection.cursor()

## Create cells table

In [3]:
# Create table of perturbations
command = '''
CREATE TABLE cells
(
    cell_uid INTEGER PRIMARY KEY AUTOINCREMENT,
    cell_id TEXT UNIQUE NOT NULL,
    cell_histology TEXT,
    cell_lineage TEXT,
    cell_source TEXT,
    cell_source_id TEXT,
    cell_type TEXT,
    gender TEXT,
    is_from_metastasis TEXT,
    metastatic_site TEXT
);
'''
cursor.execute(command)

cells_columns = [col[1] for col in cursor.execute('PRAGMA table_info(cells);')][1:]
cells_columns

['cell_id',
 'cell_histology',
 'cell_lineage',
 'cell_source',
 'cell_source_id',
 'cell_type',
 'gender',
 'is_from_metastasis',
 'metastatic_site']

In [4]:
cell_df = pandas.read_table('data/cellinfo/cellinfo.tsv.gz', compression='gzip', na_values=['-666', ''])
cell_df = cell_df[cells_columns]
cell_df.dropna(subset=['cell_id'], inplace=True)
cell_df.head()

,cell_id,cell_histology,cell_lineage,cell_source,cell_source_id,cell_type,gender,is_from_metastasis,metastatic_site
0,HEK293T,embryonal kidney,kidney,NaN,NaN,NaN,NaN,NaN,NaN
1,293,embryonal kidney,NaN,DSMZ,ACC305,NaN,NaN,NaN,NaN
2,380,leukemia,"haematopoietic,lymphoid",DSMZ,ACC39,NaN,NaN,NaN,NaN
3,813,lymphoblastoid,NaN,ECACC,85100105,NaN,NaN,NaN,NaN
4,862,"carcinoma,large cell",lung,NaN,NaN,NaN,NaN,N,NaN


In [5]:
# Create table of perturbations
cell_df.to_sql('cells', connection, if_exists='append', index=False)

## Create perts table

In [6]:
# Create table of perturbations
command = '''
CREATE TABLE perts
(
    pert_uid INTEGER PRIMARY KEY AUTOINCREMENT,
    pert_id TEXT UNIQUE NOT NULL,
    pert_iname TEXT,
    pert_type TEXT NOT NULL,
    num_gold INTEGER NOT NULL,
    num_inst INTEGER NOT NULL,
    num_sig INTEGER NOT NULL,
    in_summly INTEGER NOT NULL,
    inchi_string TEXT,
    inchi_key TEXT,
    pubchem_cid INTEGER
);
'''
cursor.execute(command)

perts_columns = [col[1] for col in cursor.execute('PRAGMA table_info(perts);')][1:]
perts_columns

['pert_id',
 'pert_iname',
 'pert_type',
 'num_gold',
 'num_inst',
 'num_sig',
 'in_summly',
 'inchi_string',
 'inchi_key',
 'pubchem_cid']

In [7]:
# read L1000 perturbagens
pert_df = pandas.read_table('data/pertinfo/pertinfo.tsv.gz', compression='gzip', na_values=['-666', ''])
pert_df = pert_df[perts_columns]
pert_df['pubchem_cid'] = pert_df.pubchem_cid.astype(int, raise_on_error=False)
pert_df['in_summly'] = pert_df.in_summly.map(lambda x: x is True)
pert_df['inchi_key'] = pert_df.inchi_key.str.replace('InChIKey=', '')
pert_df.head()

,pert_id,pert_iname,pert_type,num_gold,num_inst,num_sig,in_summly,inchi_string,inchi_key,pubchem_cid
0,CSS001-ATTGCAT,ATTGCAT,trt_sh.css,0,0,7,False,NaN,NaN,NaN
1,CSS001-GAGGATA,GAGGATA,trt_sh.css,0,0,1,False,NaN,NaN,NaN
2,CSS001-TCAATGA,TCAATGA,trt_sh.css,0,0,7,False,NaN,NaN,NaN
3,CSS001-TCAGTTC,TCAGTTC,trt_sh.css,0,0,7,False,NaN,NaN,NaN
4,CSS001-TCCATCA,TCCATCA,trt_sh.css,0,0,1,False,NaN,NaN,NaN


In [8]:
# Create table of perturbations
pert_df.to_sql('perts', connection, if_exists='append', index=False)

## Create sigs tables

In [9]:
# Create table of perturbations
command = '''
CREATE TABLE sigs
(
    sig_uid INTEGER PRIMARY KEY AUTOINCREMENT,
    sig_id TEXT UNIQUE NOT NULL,
    pert_id TEXT NOT NULL,
    pert_itime TEXT,
    pert_idose TEXT,
    cell_id TEXT NOT NULL,
    is_gold INTEGER NOT NULL,
    ngenes_modulated_dn_lm INTEGER NOT NULL,
    ngenes_modulated_up_lm INTEGER NOT NULL,
    FOREIGN KEY(pert_id) REFERENCES perts(pert_id),
    FOREIGN KEY(cell_id) REFERENCES cells(cell_id)
);
'''
cursor.execute(command)

sig_columns = [col[1] for col in cursor.execute('PRAGMA table_info(sigs);')][1:]
sig_columns

['sig_id',
 'pert_id',
 'pert_itime',
 'pert_idose',
 'cell_id',
 'is_gold',
 'ngenes_modulated_dn_lm',
 'ngenes_modulated_up_lm']

In [10]:
# read L1000 perturbagens
sig_df = pandas.read_table('data/siginfo/siginfo.tsv.gz', compression='gzip', na_values=['-666', ''])
sig_df = sig_df[sig_columns]
sig_df.head()

,sig_id,pert_id,pert_itime,pert_idose,cell_id,is_gold,ngenes_modulated_dn_lm,ngenes_modulated_up_lm
0,CVD001_HUH7_24H:BRD-A13020530-001-01-7:10,BRD-A13020530,24 h,10 µM,HUH7,False,311,260
1,CVD001_HUH7_24H:BRD-K07762753-001-03-6:50,BRD-K07762753,24 h,50 µM,HUH7,True,298,302
2,CPC004_PC3_6H:BRD-K34820100-001-02-1:10,BRD-K34820100,6 h,10 µM,PC3,False,41,39
3,CPC004_PC3_6H:BRD-A22844106-001-16-1:10,BRD-A22844106,6 h,10 µM,PC3,False,14,12
4,CPC004_PC3_6H:BRD-A55393291-001-05-7:10,BRD-A55393291,6 h,10 µM,PC3,False,8,6


In [11]:
# Create table of perturbations
sig_df.to_sql('sigs', connection, if_exists='append', index=False)

## Similarity

In [12]:
# Create table of compound similarity
command = '''
CREATE TABLE similarities
(
    pert_uid_0 INTEGER NOT NULL,
    pert_uid_1 INTEGER NOT NULL,
    chemical REAL,
    -- FOREIGN KEY(pert_uid_0) REFERENCES perts(pert_uid),
    -- FOREIGN KEY(pert_uid_1) REFERENCES perts(pert_uid),
    PRIMARY KEY(pert_uid_0, pert_uid_1)
);
'''
cursor.execute(command);

## Commit and close database

In [13]:
connection.commit()
connection.close()